In [263]:
import sys
import importlib
sys.path.append('..')  # Go up one directory to where utils.py is located
import utils  # Ensure you import the module itself, not specific items
importlib.reload(utils)  # Reload the module to apply updates
from utils import *

In [264]:
def analytic_solution(iv, t):

    f0 = iv[:,:,0,None,0]
    g0 = iv[:,:,0,None,1]    
    
    f = iv[:,:,1:,0]
    g = iv[:,:,1:,1]
    
        
    
    k = np.arange(1, f.shape[1]+1)[None,None,:]
   
    
    f = f * np.cos(2 * np.pi * k * t) + g * np.sin(2 * np.pi * k * t) / (2 * np.pi * k)
    g = g * np.cos(2 * np.pi * k * t) - f * np.sin(2 * np.pi * k * t) * (2 * np.pi * k)
    
    f0 = f0 + g0 * t
    g0 = g0 * np.ones_like(t)
    
    
    f = np.concat((f0, f), axis=2)
    g = np.concat((g0, g), axis=2)
    

    y = np.stack((f, g), axis=-1)
    
    return y
        
    


In [265]:
def get_data(num_iv=100, num_t=10,  N=10, Tmax=0.1):
    iv = np.random.uniform(low=-1, high=1, size=(num_iv, 1, N, 2)).astype(np.float32)

    if num_t > 1:
        t = np.random.uniform(low=0, high=Tmax, size=(1, num_t, 1)).astype(np.float32)
    else:
        t = np.array([Tmax])[:,None].astype(np.float32)


    y = analytic_solution(iv, t)
    X = (iv[:,0,:,:], t[0,:,:])
    
    return X, y

In [266]:
X_train, y_train = get_data()
X_test, y_test = get_data()

In [267]:
net = DeepONet([40, 40, 40], [40, 40, 40], activation='htanh', N=10, scheme=2)
model = Model(X_train, y_train, X_test, y_test, net)
model.train(iterations=10_000)
model.plot_losshistory()

Step 	 Train loss 	 Test loss
torch.Size([100, 10, 2])
torch.Size([20, 20])


RuntimeError: einsum(): subscript d has size 20 for operand 1 which does not broadcast with previously seen size 2